# Mode Share Results Script
#### PC (creds to BC)
This script generates trips to/from based on a zone mapping matrix using the Microsim outputs

In [39]:
import os
import glob
import pandas as pd
import numpy as np
from copy import deepcopy

In [40]:
# where your runs are
run_directory = r"\\tore-infs01\Models\VaughanTMP\Run Outputs"

# where your run folder is
run_folder = r"20191211_KK"
run_path = os.path.join(run_directory,run_folder)

# where your zonemapping file is
input_pth = r"C:\Users\pechen\Desktop\VTP\Zones"
zone_map = pd.read_csv(os.path.join(input_pth,"VTP_ZoneMapping.csv"))
zone_map_min = zone_map

In [41]:
microsim_location = r"Microsim Results"
microsim_path = os.path.join(run_path, microsim_location)

persons_df = pd.read_csv(os.path.join(microsim_path,"persons.csv"))
trips_df = pd.read_csv(os.path.join(microsim_path,"trips.csv"))
tripmode_df = pd.read_csv(os.path.join(microsim_path,"trip_modes.csv"))
trips_gta = trips_df[(trips_df.o_zone < 6000) & (trips_df.d_zone < 6000)]

trips_gta = trips_gta.merge(persons_df,on=["household_id","person_id"])
trips_gta = trips_gta[trips_gta.age >=11]

trips_gta_time = tripmode_df.merge(trips_gta,how = "right", on = ['household_id', 'person_id', 'trip_id'])

trips_gta_time["weight"] = trips_gta_time["weight"]/10
trips_gta_time["weight_z"] = trips_gta_time["weight"]*trips_gta_time["weight_x"]

ms_daily_GTA = pd.pivot_table(trips_gta_time,columns = "mode", values = "weight_z", aggfunc = sum)

In [42]:
trips_407 = trips_df
trips_407  = trips_407.merge(persons_df,on=["household_id","person_id"])
trips_407 = trips_407[trips_407.age >=11]
trips_407 = trips_407.merge(zone_map_min,left_on = "o_zone", right_on ="Zone").drop(columns = "Zone").rename(columns = {"Region":"region_o"})
trips_407 = trips_407.merge(zone_map_min,left_on = "d_zone", right_on ="Zone").drop(columns = "Zone").rename(columns = {"Region":"region_d"})

trips_407_time = tripmode_df.merge(trips_407,how = "right", on = ['household_id', 'person_id', 'trip_id'])
trips_407_time["weight"] = trips_407_time["weight"]/10
trips_407_time["weight_z"] = trips_407_time["weight"]*trips_407_time["weight_x"]
trips_407_am = trips_407_time[(trips_407_time.o_depart >= 360) & (trips_407_time.o_depart < 540)]
trips_407_pm = trips_407_time[(trips_407_time.o_depart >= 900) & (trips_407_time.o_depart < 1140)]

In [43]:
daily_modeshare = pd.pivot_table(trips_407_time,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)
am_modeshare = pd.pivot_table(trips_407_am,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)
pm_modeshare = pd.pivot_table(trips_407_pm,index = ["region_o","region_d"],columns = "mode", values = "weight_z", aggfunc = sum,margins = True).fillna(0)

In [44]:
am_modeshare.to_csv(os.path.join(input_pth,"am_modeshare_2016_bau.csv")) #Time Period MS
pm_modeshare.to_csv(os.path.join(input_pth,"pm_modeshare_2016_bau.csv"))
daily_modeshare.to_csv(os.path.join(input_pth, "daily_modeshare_2016_bau.csv"))

In [38]:
trips_df

,household_id,person_id,trip_id,o_act,o_zone,d_act,d_zone,weight
0,1,1,1,Home,1,IndividualOther,5,1
1,9,1,1,Home,1,Market,252,1
2,10,1,1,Home,1,IndividualOther,38,1
3,11,1,1,Home,1,IndividualOther,5,1
4,6,1,1,Home,1,PrimaryWork,220,1
...,...,...,...,...,...,...,...,...
22642822,4559527,2,2,JointOther,5198,Home,5248,1
22642823,4559527,2,3,Home,5248,IndividualOther,5198,1
22642824,4559527,2,4,IndividualOther,5198,Home,5248,1
22642825,4559528,1,1,Home,5248,PrimaryWork,4080,1
